#### Defined by User

In [ ]:
## Define the model monitor name here that you will be creating 
MONITOR_NAME = "mm-demo"
## Specify the data source [aws_s3,sql,local]
DATA_SOURCE = "local"
## Specify Training type, [training,retraining]
INPUT_TRAIN_TYPE = "training"
## DKUBE USERNAME
DKUBEUSERNAME = ""

## Dkube information
import os
TOKEN = os.getenv("DKUBE_USER_ACCESS_TOKEN","")
DKUBE_URL = os.getenv("DKUBE_URL","")

## AWS_S3 
ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID","")
SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY","")

## SQL
HOSTNAME = os.getenv("HOSTNAME","")
DATABASENAME= os.getenv("DATABASENAME","")
DBUSERNAME = os.getenv("DBUSERNAME","")
PASSWORD = os.getenv("PASSWORD","")



if TOKEN == '' or DKUBEUSERNAME == '' or DKUBE_URL == '':
    print("Please fill the Dkube details first (TOKEN, DKUBE_URL, DKUBEUSERNAME)")
    raise TypeError
if DATA_SOURCE == 'aws_s3' and (ACCESS_KEY == '' or SECRET_KEY == ''):
    print("Please fill the AWS_S3 details first (ACCESS_KEY, SECRET_KEY)")
    raise TypeError
if DATA_SOURCE == 'sql' and (HOSTNAME == '' or DATABASENAME == '' or DBUSERNAME == '' or PASSWORD == ''):
    print("Please fill the SQL details first (HOSTNAME, DATABASENAME, DBUSERNAME, PASSWORD)")
    raise TypeError
    


In [ ]:
%store MONITOR_NAME DATA_SOURCE INPUT_TRAIN_TYPE DKUBEUSERNAME TOKEN DKUBE_URL ACCESS_KEY SECRET_KEY HOSTNAME DATABASENAME DBUSERNAME PASSWORD

#### Dkube Resources

In [ ]:
import time,json,shutil
from dkube.sdk import *

In [ ]:
api = DkubeApi(URL=DKUBE_URL,token=TOKEN)
if DKUBEUSERNAME == api.validate_token()['username']:
    pass
else:
    print("Invalid User, please check your username, first")

#### Dataset

In [ ]:
try:
    dataset = DkubeDataset(DKUBEUSERNAME, name='insurance-data')
    dataset.update_git_details(url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv")
    dataset.update_dataset_source(source="pub_url")
    api.create_dataset(dataset)

except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

### Model Monitor Datasets

##### (Training / Retraining) Dataset

In [ ]:
try:
    dataset = DkubeDataset(DKUBEUSERNAME, name='insurance-training-data')
    api.create_dataset(dataset)

except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

##### Predict Dataset

In [ ]:
MONITOR_NAME
print(os.getenv("AWS_ACCESS_KEY_ID",ACCESS_KEY))
print(os.getenv("AWS_SECRET_ACCESS_KEY",SECRET_KEY))

In [ ]:
pname =  MONITOR_NAME+'-predict'
prefix = MONITOR_NAME+'/predict'

if DATA_SOURCE == 'local':
    try:
        dataset = DkubeDataset(DKUBEUSERNAME, name=pname)
        api.create_dataset(dataset)
    
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
            

if DATA_SOURCE == 'aws_s3':
    try:
        dataset = DkubeDataset(DKUBEUSERNAME, name=pname,remote=True)
        dataset.update_dataset_source('aws_s3')
        dataset.update_awss3_details(bucket="mm-workflow",prefix=prefix,key=os.getenv("AWS_ACCESS_KEY_ID",ACCESS_KEY),secret=os.getenv("AWS_SECRET_ACCESS_KEY",SECRET_KEY))
        api.create_dataset(dataset)
        
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    

##### Labelled Dataset

In [ ]:
gname = MONITOR_NAME+'-groundtruth'
prefix = MONITOR_NAME+'/groundtruth'

if DATA_SOURCE == 'local':
    try:
        dataset = DkubeDataset(DKUBEUSERNAME, name=gname)
        api.create_dataset(dataset)
    
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
            

if DATA_SOURCE == 'aws_s3':
    try:
        dataset = DkubeDataset(DKUBEUSERNAME, name=gname,remote=True)
        dataset.update_dataset_source('aws_s3')
        dataset.update_awss3_details(bucket="mm-workflow",prefix=prefix,key=os.getenv("AWS_ACCESS_KEY_ID",ACCESS_KEY),secret=os.getenv("AWS_SECRET_ACCESS_KEY",SECRET_KEY))
        api.create_dataset(dataset)
        
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")

#### Model

In [ ]:
if api.dkubeinfo['features'] == ['monitoring']:
    try:
        model = DkubeModel(DKUBEUSERNAME, name='insurance-model')
        model.update_model_source(source='git')
        model.update_git_details('https://github.com/oneconvergence/dkube-examples/blob/monitoring/insurance/model.joblib',branch="monitoring")
        api.create_model(model)

    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
else:
    try:
        model = DkubeModel(DKUBEUSERNAME, name='insurance-model')
        model.update_model_source(source='dvs')
        api.create_model(model)
    
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")

#### Cleanup

In [ ]:
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False
if CLEANUP:
    api.delete_code(DKUBEUSERNAME,'insurance',force=True)
    api.delete_dataset(DKUBEUSERNAME,'insurance-data',force=True)
    api.delete_dataset(DKUBEUSERNAME,'insurance-training-data',force=True)
    api.delete_dataset(DKUBEUSERNAME,pname,force=True)
    api.delete_dataset(DKUBEUSERNAME,gname,force=True)
    api.delete_model(DKUBEUSERNAME,'insurance-model',force=True)
    %store -d MONITOR_NAME DATA_SOURCE INPUT_TRAIN_TYPE DKUBEUSERNAME TOKEN DKUBE_URL ACCESS_KEY SECRET_KEY HOSTNAME DATABASENAME DBUSERNAME PASSWORD  
    